# Web Scrapping on Flipkart.com for Laptops Data

### Step 1: Importing all necessary libraries.

In [1]:
import requests 
import bs4
from bs4 import BeautifulSoup
import re
import io

### Step 2: Send Request for data and getting the Response.

In [2]:
response = requests.get("https://www.flipkart.com/search?q=laptop&sid=6bo%2Cb5g&as=on&as-show=on&otracker=AS_QueryStore_OrganicAutoSuggest_0_3_na_na_pr&otracker1=AS_QueryStore_OrganicAutoSuggest_0_3_na_na_pr&as-pos=0&as-type=RECENT&suggestionId=laptop%7Cin+Laptops&requestId=2ef01484-77d1-4efb-a2fe-83792a72c6b4&as-backfill=on")
print(response)
requests.session().close()

<Response [200]>


### Step 3 : Getting the count for no. of times loop to be executed from no. of products we get in response.

In [5]:
soup = BeautifulSoup(response.content , "html.parser")
count = soup.find_all("span", attrs = {"class":"_2yAnYN"})

# print(count[0].getText())

r = re.compile("[0-9]+")
count = (re.findall(r , count[0].getText())[-1]).split(",")
s = ""
count = int(s.join(count))
print(count)                  # Total no. of products.
count = count // 24           # Here 24 is the no. of products on per page.

print(count)                  # Finally the count for loop to be executed.
print(type(count))


926
38
<class 'int'>


### Step 4: Create a .csv file to store data.

In [6]:
with io.open("laptopsData.csv","w",encoding="utf8") as f1:
    f1.write("Title,Ram,Hard_disk,Display,Operating_system,Processor,Selling_price,Cost_price,Rating,Discount\n")

### Step 5: Some functions we needed to clean the data.

In [7]:
def price(str):               # this function returns a price by removing ",'s" & " ₹ " this sign.
    s = ""
    l = str.split(",")
    for i in l:
        s += i
    return s[1:]
print(price('₹57,990'))

def processors(str):          # this function returns str by replacing ",'s" with " | ".
    s = ""
    if ',' in str:
        l = str.split(",")
        for i in l:
            s += i + "|"
        return s
    else:        
        return str
print(processors('Redmi 8 (Onyx Black, 64 GB)'))

57990
Redmi 8 (Onyx Black| 64 GB)|


### Step 6: Final stage of storeing data into the csv file.

In [9]:

for i in range (count + 1):
    i += 1
    url1 = "https://www.flipkart.com/search?q=laptop&sid=6bo%2Cb5g&as=on&as-show=on&otracker=AS_QueryStore_OrganicAutoSuggest_0_3_na_na_pr&otracker1=AS_QueryStore_OrganicAutoSuggest_0_3_na_na_pr&as-pos=0&as-type=RECENT&suggestionId=laptop%7Cin+Laptops&requestId=2ef01484-77d1-4efb-a2fe-83792a72c6b4&as-backfill=on&page="
    url1 += str(i)
    resp = requests.get(url1)
    requests.session().close()
    readfull = BeautifulSoup(resp.content , "html.parser")


    cards = readfull.find_all("div",attrs={"class":"_1UoZlX"})
    for card in cards:
        disp = ""
        proc =""
        os =""
        ram = ""
        hd =""
        discount =""
        title = card.find("div", attrs={"class":"_3wU53n"}).getText()
        Selling_price = card.find("div",attrs={"class":"_2rQ-NK"}).getText()
        Cost_price = card.find("div",attrs={"class":"_2GcJzG"})
        discount = card.find("div",attrs={"class":"VGWI6T"})
        if Cost_price == None:
            Cost_price = Selling_price
        else:
            Cost_price = Cost_price.getText()
        rating = card.find("div",attrs={"class":"hGSR34"})
        if rating == None:
            rating = ""
        else:
            rating = rating.getText()

        if discount == None:
            discount = ""
        else:
            discount = discount.getText()
            
        j = card.find("ul", attrs = {"class":"vFw0gD"})
        specs = j.find_all("li",attrs = {"class":"tVe95H"})
        for k in specs:
            if k.getText().split(" ")[-1] == "Display":
                disp = k.getText()
            elif k.getText().split(" ")[-1] == "Processor" or "Processor" in k.getText().split(" ") :
                proc = k.getText()
            elif k.getText().split(" ")[-1] == "RAM":
                ram = k.getText()
            elif k.getText().split(" ")[-1] == "System":
                os = k.getText()
            elif k.getText().split(" ")[-1] == "HDD" or k.getText().split(" ")[-1] == "SSD":
                hd = k.getText()
#         "Title,Ram,Hard_disk,Display,Operating_system,Processor,Selling_price,Cost_price,Rating,Discount\n"
        data=title +","+ ram +","+ hd +","+ disp +","+ os +","+ proc +","+ price(Selling_price) +","+ price(Cost_price) +","+ rating +","+ discount +"\n"
        with io.open("laptopsData.csv","a",encoding="utf8") as f1:
            f1.write(data)
#         print(disp,proc,os,ram,hd,sep=" : ")

#         print(title , Selling_price , Cost_price , rating , discount , sep=":")
print("done")        

done


In [10]:
import pandas as pd
df = pd.read_csv('laptopsData.csv')
df.head()

,Title,Ram,Hard_disk,Display,Operating_system,Processor,Selling_price,Cost_price,Rating,Discount
0,HP 14q APU Dual Core A9 - (4 GB/256 GB SSD/Win...,4 GB DDR4 RAM,256 GB SSD,35.56 cm (14 inch) Display,64 bit Windows 10 Operating System,AMD APU Dual Core A9 Processor,19990,24426,4.1,18% off
1,Lenovo Ideapad 130 Core i3 7th Gen - (4 GB/1 T...,4 GB DDR4 RAM,1 TB HDD,39.62 cm (15.6 inch) Display,64 bit Windows 10 Operating System,Intel Core i3 Processor (7th Gen),26990,33040,4.0,18% off
2,HP 15q APU Dual Core A9 - (4 GB/1 TB HDD/Windo...,4 GB DDR4 RAM,1 TB HDD,39.62 cm (15.6 inch) Display,64 bit Windows 10 Operating System,AMD APU Dual Core A9 Processor,22990,27451,4.1,16% off
3,MSI Modern 14 Core i5 10th Gen - (8 GB/512 GB ...,8 GB DDR4 RAM,512 GB SSD,35.56 cm (14 inch) Display,64 bit Windows 10 Operating System,Intel Core i5 Processor (10th Gen),49990,69990,4.6,28% off
4,Apple MacBook Air Core i5 5th Gen - (8 GB/128 ...,8 GB DDR3 RAM,128 GB SSD,33.78 cm (13.3 inch) Display,64 bit Mac OS Operating System,Intel Core i5 Processor (5th Gen),64990,84900,4.7,23% off


In [ ]:
data = pd.read_csv('')